# 2.4 用户注册与登录`[01]`(基本结构)

一般留言板都会要收集用户信息,之前的版本中因为没有用户信息我们使用用户自己写名字的方式,这种明显是很不靠谱的,那么怎么做用户登录呢,首先从数据库的改造开始:

我们把之前的一张表拆成两张,一张是Users,一张是Messages,以及一张users,messages间的关系表.

User:

字段|类型
---|---
id|int[pr]
name|String(64)
password|String(64)
email|String(64)
role_id|int[FK]

Message:

字段|类型
---|---
id|int[pr]
content|text
timestemp|datetime[idx]
author_id|int[FK]


关于用户,明显还有一张表就是用户权限表,用这个来保证不是所有人都可以操作网站.

Role:

字段|类型
---|---
Roleid|int[pr]
name|String(64)[pr]




然后我们需要给一个注册页面来让用户注册,当然配套的也要有注销的操作,要保存用户的登录信息,最简单的方式就是使用cookie,用户的密码信息最好先加密.我们使用[Flask-Bcrypt](http://flask-bcrypt.readthedocs.org/en/latest/)来加密,Flask-Bcrypt可以自定义一个数值然后通过多次迭代产生密码的散列,相对来说还是比较安全.

> 设置config,添加BCRYPT设置,新增管理员数据

In [75]:
%%writefile ../codes/msgboard/config.py
#--*--coding:utf-8 --*--
from __future__ import absolute_import,division,print_function,unicode_literals

import os
basedir = os.path.abspath(os.path.dirname(__file__))

class Config:
    SECRET_KEY = os.environ.get('SECRET_KEY') or 'hard to guess string' 
    BCRYPT_LEVEL = 10 # 配置Flask-Bcrypt拓展
    @staticmethod
    def init_app(app):
        pass
    

class DevelopmentConfig(Config): 
    SQLALCHEMY_DATABASE_URI = 'sqlite:///' + os.path.join(basedir, 'data.sqlite')
    DEBUG = True
         
class TestingConfig(Config): 
    TESTING = True
    
class ProductionConfig(Config):
    pass

config = {
    'development': DevelopmentConfig,
    'testing': TestingConfig, 
    'production': ProductionConfig,
    'default': DevelopmentConfig
}

AdminAccount = [{"name":"hsz",
                 "email":"hsz1273327@sina.cn",
                 "password":"hsz881224"
        }] 

BASE_URL = "127.0.0.1:5000"

Overwriting ../codes/msgboard/config.py


> app主体

新增注册,登录等页面,主页将只作为一个过渡如果我们的用户已经登录,那么直接进入我们的留言板,如果之前退出登录了那么会进如signin页面,如果是第一次过来我们才会把它指向主页,主页和signup页都可以实现注册.我们还要使用[Flask-Login](http://docs.jinkan.org/docs/flask-login/)来做用户登录验证.

In [184]:
%%writefile ../codes/msgboard/app.py
#--*--coding:utf-8 --*--
from __future__ import absolute_import,division,print_function,unicode_literals
"""
A message board appliation.

Author:Huang Sizhe
Date:22/01/2016
License:MIT
======================================

留言板应用

作者:黄思喆
日期:2016年1月22日
本应用使用MIT许可证

"""
from datetime import datetime,timedelta, timezone
#=================导入模块=================
from flask import Flask,render_template,make_response,redirect,url_for,flash,request

from flask.ext.bootstrap import Bootstrap

from flask_wtf.csrf import CsrfProtect
from flask.ext.wtf import Form
from wtforms import StringField, SubmitField ,PasswordField
from wtforms.validators import DataRequired, Email
#导入ORM模块
from flask.ext.sqlalchemy import SQLAlchemy
#密码加密
from flask.ext.bcrypt import Bcrypt,generate_password_hash 
from sqlalchemy.ext.hybrid import hybrid_property
#login
from flask.ext.login import UserMixin,LoginManager,login_user,logout_user,login_required,current_user


#=================载入插件=================
bootstrap = Bootstrap()
csrf = CsrfProtect()

db = SQLAlchemy()# 实例化ORM对象

bcrypt = Bcrypt()#加密
login_manager = LoginManager()


#=================应用设置=================
from config import config,BASE_URL
def create_app(config_name):
    app = Flask(__name__)
    app.config.from_object(config[config_name])
    config[config_name].init_app(app)
    bootstrap.init_app(app)
    csrf.init_app(app)
    
    db.init_app(app)#初始化数据库
    bcrypt.init_app(app)#加密
    #login
    login_manager.init_app(app)
    login_manager.login_view =  "signin"
    return app

import os
app = create_app(os.getenv('FLASK_CONFIG') or 'default') 

#flask-login的回调函数,实现后可以使用current_user来代理访问以登录的用户
@login_manager.user_loader
def load_user(userid):
    return User.query.filter(User.id == userid).first()

#================主体=====================

#---------------自定义表单验证--------------

from wtforms.validators import ValidationError

class Unique(object):
    def __init__(self, model, field, message=u'Already exist !'):
        self.model = model
        self.field = field
        self.message = message

    def __call__(self, form, field):
        check = self.model.query.filter(self.field == field.data).first()
        if check:
            raise ValidationError(self.message)

#----------------自定义过滤器--------------



#-----------------数据库对象---------------

class Role(db.Model):
    __tablename__ = 'role'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64), unique=True)
    user = db.relationship('User', backref='role')
    
    def __repr__(self):
        return '''<Role: {id} name: {name}>'''.format(id = self.id,
                                  name = self.name)
    
#user用于登录还要继承UserMixin ,这样就不用自己写几个验证函数了
class User(UserMixin,db.Model):
    __tablename__ = 'user'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64), unique=True)
    _password = db.Column(db.String(64), unique=True)
    email = db.Column(db.String(64), unique=True)
    role_id = db.Column(db.Integer, db.ForeignKey('role.id'))

    message = db.relationship('Message', backref='user')
    
    #密码的写法:
    @hybrid_property
    def password(self):
        return self._password
    
    @password.setter
    def _set_password(self, plaintext):
        self._password = bcrypt.generate_password_hash(plaintext)
    

    def __repr__(self):
        return '''<User: {id} 
        name: {name}
        role_id: {role_id}
        email: {email}>'''.format(id = self.id,
                                  name = self.name,
                                 role_id = self.role_id,
                                 email = self.email)
    def is_correct_password(self, plaintext):
        if bcrypt.check_password_hash(self._password, plaintext):
            return True

        return False
    
class Message(db.Model):
    __tablename__ = 'message'
    id = db.Column(db.Integer, primary_key=True)
    content = db.Column(db.Text, unique=True)
    timestamp = db.Column(db.DateTime, index=True, default=datetime.utcnow)
    author_id = db.Column(db.Integer, db.ForeignKey('user.id'))
    
    def __repr__(self):
        return '<message: {id} - {time} - {content}>'.format(id = self.id,
                                                             time=self.timestamp,
                                                             content = self.content)


    
##----------------主页--------------------------
class SignUp_Form(Form):
    email = StringField('Your e-mail', 
                        validators=[DataRequired(),
                                    Email(), 
                                    Unique(User, 
                                           User.email, 
                                           message='This e-mail has been used. Please use another one.')])
                        
    username = StringField('Pickup a username', validators=[DataRequired()])
    password = PasswordField('Password', validators=[DataRequired()])
    submit = SubmitField('Sign Up for MSG Board')

@app.route('/',methods = ["GET","POST"])
def index():
    
    if current_user.is_authenticated:
        return redirect(url_for("msgboard"))
    if current_user.is_active:
        return redirect(url_for("signin"))
    form = SignUp_Form()
    if form.validate_on_submit():
        user = User(name = form.username.data,
                    password = form.password.data,
                    email = form.email.data,
                    role_id = Role.query.filter_by(name='User').first().id)
        db.session.add(user)
        db.session.commit()
        return redirect(url_for('index'))
    
    return render_template('app/index.html', form=form)
##----------------msgboard页--------------------

@app.template_filter('AuthorName')
def AuthorName(text):
    return User.query.filter_by(id=int(text)).first().name

@app.template_filter('LocalTime')
def LocalTime(text):
    #return text.replace(tzinfo=timezone.utc).astimezone(timezone(timedelta(hours=8)))
    local_timezone = {"zh-cn":8}
    return text.replace(tzinfo=timezone.utc).\
               astimezone(timezone(timedelta(hours=local_timezone.get(request.headers["Accept-Language"],0))))\
    .strftime('%a, %b %d %H:%M')+" in utc+"+str(local_timezone.get(request.headers["Accept-Language"],0))
    

class MsgForm(Form):
    msg = StringField('The msg', validators=[DataRequired()])
    submit = SubmitField('Submit')


@app.route('/msgboard',methods = ["GET","POST"])
@login_required
def msgboard():
    msgform = MsgForm()
    if msgform.validate_on_submit():
        
        msg = Message(content = msgform.msg.data,
                      author_id=current_user.id)
        db.session.add(msg)
        db.session.commit()
        msgform.msg.data = ''
        return redirect(url_for('msgboard'))
        
    response = make_response(render_template('app/msgboard.html',
                                             msgform=msgform,
                                             MSG = Message.query.all()))
    return response

##----------------注册页--------------------v


@app.route("/signup",methods = ["GET","POST"])
def signup():
    form = SignUp_Form()
    if form.validate_on_submit():
        user = User(name = form.username.data,
                    password = form.password.data,
                    email = form.email.data,
                    role_id = Role.query.filter_by(name='User').first().id)
        db.session.add(user)
        db.session.commit()
        return redirect(url_for('index'))

    return render_template('app/signup.html', form=form)


##----------------登录页--------------------v
                        


class SignIn_Form(Form):
    email = StringField('Your e-mail', validators=[DataRequired()])
    password = PasswordField('Password', validators=[DataRequired()])
    submit = SubmitField('Sign In')
@app.route("/signin",methods = ["GET","POST"])
def signin():
    form = SignIn_Form()

    if form.validate_on_submit():
        email = User.query.filter_by(email=form.email.data).first_or_404()
        if email.is_correct_password(form.password.data):
            login_user(email)

            return redirect(url_for('index'))
        else:
            return redirect(url_for('signin'))
    return render_template('app/signin.html', form=form)

##---------------退出登录--------------------v
                        
@app.route('/signout')
@login_required
def signout():
    logout_user()
    flash('You have been signed out.')
    return redirect(url_for('index'))

Overwriting ../codes/msgboard/app.py


>index.html

In [179]:
%%writefile ../codes/msgboard/templates/app/index.html
{% extends "/base.html" %}
{% import "bootstrap/wtf.html" as wtf %}
{% block title %}Sign Up Page{% endblock %}
{% block content %}
<div id="content">
    <div class="container">
        <div class="page-header">
            <h1>Welcome to this application!</h1>
        </div>      
        <div class="page-body">
            {{ wtf.quick_form(form,form_type='horizontal') }}
            <p class="pull-right">you already have a account? please <a href="{{ url_for('signin') }}"}>sign in</a></p>
        </div> 
        <div id=img>
             <img src="{{ url_for('static', filename = 'jftw.jpg') }}"></img>
        </div>
    </div>    
</div>   
{{ super() }}
{% endblock %}

Overwriting ../codes/msgboard/templates/app/index.html


> msgboard.html

In [164]:
%%writefile ../codes/msgboard/templates/app/msgboard.html
{% extends "/base.html" %}
{% import "bootstrap/wtf.html" as wtf %}
{% block title %}MSG BOARD{% endblock %}
{% block content %}

    <div class="container">
        <div class="page-header">
            <h1>MSG BOARD!</h1>
            {{ wtf.quick_form(msgform,form_type='horizontal') }}
            
            <ul class=entries>
              {% for entry in MSG %}
                <li><h4>{{ entry.author_id|AuthorName}} said at {{entry.timestamp|LocalTime }}:</h4><p>{{ entry.content }}<p></li>
              {% else %}
                <li><em>Unbelievable.  No messages here so far</em></li>
              {% endfor %}
            </ul>
            
        </div>      
    </div>    
    
{{ super() }}
{% endblock %}

Overwriting ../codes/msgboard/templates/app/msgboard.html


> signup.html

In [165]:
%%writefile ../codes/msgboard/templates/app/signup.html
{% extends "/base.html" %}
{% import "bootstrap/wtf.html" as wtf %}
{% block title %}Sign Up Page{% endblock %}
{% block content %}
<div id="content">
    <div class="container">
        <div class="page-header">
            <h1>Sign Up Page</h1>
        </div>      
        <div class="page-body">
            {{ wtf.quick_form(form,form_type='horizontal') }}
            <p class="pull-right">you already have a account? please <a href="{{ url_for('signin') }}"}>sign in</a></p>
        </div> 
       
    </div>    
</div>   
{{ super() }}
{% endblock %}


Overwriting ../codes/msgboard/templates/app/signup.html


> signin.html

In [166]:
%%writefile ../codes/msgboard/templates/app/signin.html
{% extends "/base.html" %}
{% import "bootstrap/wtf.html" as wtf %}
{% block title %}Sign In Page{% endblock %}
{% block content %}
<div id="content">
    <div class="container">
        <div class="page-header">
            <h1>SignIn Page</h1>
        </div>      
        <div class="page-body">
            {{ wtf.quick_form(form,form_type='horizontal') }}
            <p class="pull-right">you do not have a account? please click here <a href="{{ url_for('signup') }}"> to create a account</a></p>
        </div>  
    </div>    
</div>    
{{ super() }}
{% endblock %}

Overwriting ../codes/msgboard/templates/app/signin.html


> footer.html

之前的我们内容多,所以不固定高度也还行,不过现在内容少我们就得定义页脚一直保持在页面底部.

In [167]:
%%writefile ../codes/msgboard/templates/blocks/footer.html
<footer class="footer">
    <div class="container navbar-fixed-bottom">
        <p>&copy;Huang Sizhe 2015</p>
        <p>版权声明:MIT License</p>
    </div>
</footer>

Overwriting ../codes/msgboard/templates/blocks/footer.html


> mainnavbar.html

导航栏需要修改为可以根据登录状况来判断显示什么连接,注意,current_user.is_authenticated是一个布尔值,不是方法

要使导航条右对齐可以使用navbar-right

In [155]:
%%writefile ../codes/msgboard/templates/blocks/mainnavbar.html
<nav class="navbar navbar-inverse navbar-fixed-top"  role="navigation">
         <div class="container">
             <div class="navbar-header">
                 <button type="button" class="navbar-toggle" data-toggle="collapse" data-target=".navbar-collapse">
                     <span class="sr-only">切换导航</span>
                     <span class="icon-bar"></span>
                     <span class="icon-bar"></span>
                     <span class="icon-bar"></span>
                 </button>
                 <a class="navbar-brand" href="/">MyAPP</a>
             </div>
             <div class="navbar-collapse collapse">
                 <ul class="nav navbar-nav">
                    <li><a href="{{url_for('msgboard')}}">msg board</a></li>
                 </ul>
                 <ul class="nav navbar-nav navbar-right">
                    {% if current_user.is_authenticated %}{# is_authenticated判断用户是否已经登录#}
                    <li><a href="{{url_for('signout') }}">Sign Out</a></li>
                    {% else %}
                    <li><a href="{{ url_for('signin') }}">signin</a></li>
                    <li><a href="{{ url_for('signup') }}">signup</a></li>
                    {% endif %}
                 </ul>
             </div>
            
         </div>
    </nav>

Overwriting ../codes/msgboard/templates/blocks/mainnavbar.html


> manager.py

在启动脚本中把新加的数据库信息填上,并写一个初始化数据库的方法,方便初始化.

In [156]:
%%writefile ../codes/msgboard/manager.py
#--*--coding:utf-8 --*--
from __future__ import absolute_import,division,print_function,unicode_literals
"""
A startup manager of the application.

Author:Huang Sizhe
Date:22/01/2016
License:MIT
======================================

应用的启动文件

作者:黄思喆
日期:2016年1月22日
本应用使用MIT许可证

"""

__author__ = "Huang Sizhe"
__date__ = "22/01/2016"

import os
import sys
from flask.ext.script import Manager,Shell

root = os.path.dirname(__file__)
#把新加的表名放进去便于操作
from app import app,db,Message,User,Role
from config import AdminAccount

manager = Manager(app)

def init_db(db):
    db.create_all()
    admin_role = Role(name='Admin')
    mod_role = Role(name='Moderator')
    user_role = Role(name='User')
    db.session.add_all([admin_role, mod_role, user_role])
    for i in AdminAccount:
        adminaccount = User(name=i["name"] ,email=i["email"],role =admin_role ,password = i["password"])
        db.session.add(adminaccount)
    db.session.commit()


def make_shell_context():
    return dict(app=app, 
                db=db,
                Message=Message,
                User=User,
                Role=Role,
                init_db=init_db)

manager.add_command("shell", Shell(make_context=make_shell_context))

if __name__ == '__main__':
    manager.run()

Overwriting ../codes/msgboard/manager.py


In [1]:
!python3 ../codes/msgboard/manager.py runserver

Traceback (most recent call last):
  File "../codes/msgboard/manager.py", line 28, in <module>
    from app import app,db,Message,User,Role
  File "/Users/huangsizhe/workspace/post/ComputerScience/CodingLanguages/Python_Total_Tutorial/常用的第三方库/web框架/微框架(Flask)/codes/msgboard/app.py", line 34
    from flask.ext.login import UserMixin,LoginManager,login_user,logout_user,
                               ^
SyntaxError: trailing comma not allowed without surrounding parentheses


# 总结

这部分我们够建立一个可用的留言板,他有基本的用户注册,用户登录,用户注销功能,并有写留言的功能.该留言板还附带每条消息的时间戳.在下一部分我们会继续完善它的功能,包括消息的显示顺序,新增用户编辑留言功能,用户换邮箱,改密码等操作,以及邮箱验证等,还有评论功能等

用到的模块:

包|作用
---|---
flask|flask web框架
flask-script|flask的上下文shell
jinja2|flask的默认模板
flask-bootstrap|flask的bootstrap前端扩展
flask-wtf|构建表单
Flask-Bcrypt|密码加密
flask-SQLAlchemy|关系数据库ORM
Flask-Login|登录管理
